## 📌 Introduction

In this notebook, we implement baseline machine learning models to simulate a company's typical ML pipelines, specifically focusing on tasks such as Loan Approval, Fraud Detection, and Marketing Prediction. We start with a baseline scenario representing common industry practice, where multiple models are trained on extensive data without considering potential privacy vulnerabilities. This setup will later serve as a comparison point for our privacy-preserving methodology.


In [2]:
!pip install ai-privacy-toolkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 2.1 MB/s eta 0:00:00


In [3]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.1 MB/s eta 0:00:00


## 🛠 Generating Synthetic Data

We begin by generating synthetic data designed to mimic real-world company datasets. This dataset includes a variety of numerical and categorical features typically collected by businesses, such as credit scores, transaction amounts, and customer engagement metrics. Additionally, we've introduced a deliberately "leaky" feature correlated with target labels (`loan_default`) to simulate a scenario where sensitive information could inadvertently leak, highlighting potential privacy risks.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Set seed for reproducibility
np.random.seed(42)

# Number of samples
num_samples = 500

# Generate numerical features
synthetic_data = pd.DataFrame({
    "credit_score": np.random.randint(300, 850, num_samples),
    "income_bracket": np.random.choice(["Low", "Medium", "High"], num_samples),
    "transaction_amount": np.random.uniform(10, 10000, num_samples),
    "transaction_frequency": np.random.randint(1, 50, num_samples),
    "monthly_expenses": np.random.uniform(500, 5000, num_samples),
    "savings_balance": np.random.uniform(100, 100000, num_samples),
    "loan_default": np.random.choice([0, 1], num_samples),
    "fraudulent_transaction": np.random.choice([0, 1], num_samples),
    "ad_engagement_score": np.random.uniform(0, 100, num_samples)
})

# Introduce a "leaky" feature that slightly correlates with target labels
synthetic_data["leaky_feature"] = synthetic_data["loan_default"] * np.random.uniform(0.8, 1.2, num_samples)

# Save dataset
synthetic_data.to_csv("synthetic_dataset.csv", index=False)
print("✅ New synthetic dataset generated and saved as 'synthetic_dataset.csv'")


✅ New synthetic dataset generated and saved as 'synthetic_dataset.csv'


## 💾 Saving the Synthetic Dataset
Here, we save our generated synthetic dataset as a CSV file (`synthetic_dataset.csv`). This step allows for easy reuse and reproducibility of results across various experiments and model comparisons.


## ⚙️ Splitting Data into Train and Test Sets
We split our synthetic dataset into training and testing sets. Notice that we allocate 70% of data to testing. This unusually large test set size is intentional, demonstrating how limited training data might exacerbate privacy vulnerabilities, especially in scenarios such as membership inference attacks.


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)  # 70% test data


## 🌳 Training a Baseline Decision Tree Classifier
In this step, we train a baseline Decision Tree Classifier with limited complexity (`max_depth=5`). This model intentionally risks overfitting due to limited training data and model simplicity, making it vulnerable to privacy attacks like membership inference. Such a scenario represents a common vulnerability in real-world ML pipelines that disregard privacy considerations.


In [7]:
from sklearn.tree import DecisionTreeClassifier

# Train a weak, overfitted model
vulnerable_model = DecisionTreeClassifier(max_depth=5, random_state=42)
vulnerable_model.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=5, random_state=42)

## 🚨 Setting Up Membership Inference Attack
We utilize a Membership Inference Attack (MIA), specifically a black-box attack type implemented via the Adversarial Robustness Toolbox (ART). MIA attacks exploit model predictions to infer if specific data samples were part of the model's training set, demonstrating potential privacy vulnerabilities of ML models trained without privacy safeguards.


In [8]:
mia_attack = MembershipInferenceBlackBox(classifier, attack_model_type="rf")


In [9]:
dataset["leaky_feature"] = np.where(dataset.index < len(dataset) * 0.5, 1, 0)  # Half of dataset has a leakage flag


## 🛡️ Conducting Membership Inference Attack
We execute the Membership Inference Attack (MIA) to evaluate the vulnerability of our baseline model. The attack trains on the predictions from the Random Forest Classifier, attempting to accurately predict whether specific records were used in the training set.


In [20]:
# 🚀 Corrected Baseline + Membership Inference Attack Example

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from art.estimators.classification import SklearnClassifier
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox

# Step 1: Synthetic Dataset Creation (Clearly Vulnerable)
np.random.seed(42)
num_samples = 1000
X_synthetic = pd.DataFrame({
    "income": np.random.normal(50000, 5000, num_samples),
    "age": np.random.randint(20, 70, num_samples),
    "leaky_feature": np.concatenate([np.ones(num_samples//2), np.zeros(num_samples//2)])
})

y_synthetic = np.concatenate([np.ones(num_samples//2), np.zeros(num_samples//2)])

# Step 2: Train-Test Split (Vulnerable Setting)
X_train_vuln, X_test_vuln, y_train_vuln, y_test_vuln = train_test_split(
    X_synthetic, y_synthetic, test_size=0.5, random_state=42
)

# Step 3: Train Vulnerable Baseline Model
baseline_model = RandomForestClassifier(random_state=42)
baseline_model.fit(X_train_vuln, y_train_vuln)

# Step 3a: Wrap Model with ART Classifier
art_classifier_vuln = SklearnClassifier(model=baseline_model)

# Step 4: Perform Membership Inference Attack (BlackBox)
mia_attack_vuln = MembershipInferenceBlackBox(
    art_classifier_vuln,
    attack_model_type='rf'
)

mia_attack_vuln.fit(
    X_train_vuln.values, y_train_vuln,
    X_test_vuln.values, y_test_vuln
)

# Step 5: Evaluate Attack Success Rate
mia_preds_vuln_train = mia_attack_vuln.infer(X_train_vuln.values, y=y_train_vuln)
mia_preds_vuln_test = mia_attack_vuln.infer(X_test_vuln.values, y=y_test_vuln)

mia_attack_accuracy_train = np.mean(mia_preds_vuln_train)
mia_attack_accuracy_test = np.mean(mia_preds_vuln_test)

print(f"🚨 Membership Inference Attack Success: {mia_preds_vuln_train.mean():.2%}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


🚨 Membership Inference Attack Success: 82.60%


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## 📊 Evaluating Membership Inference Attack Results
We measure the effectiveness of our privacy attack by calculating the attack accuracy—the proportion of correctly inferred membership statuses. The high success rate of approximately **82.6%** indicates significant vulnerability, confirming the importance of incorporating robust privacy-preserving techniques into ML pipelines.
